https://www.sdss.org/dr15/mastar/mastar-tutorials/

https://dr15.sdss.org/sas/dr15/manga/spectro/mastar/v2_4_3/v1_0_2/



In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import glob
from astropy.table import Table
from astropy.io import fits
from os import listdir
from glob import glob
from os import getcwd, chdir
import warnings
warnings.filterwarnings('ignore')

os.system('ulimit -n 50000')
os.system('ulimit -a')

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 15
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

yInit = 0
modelWavelengthShift = 0

goodspec = fits.open("/Users/josephkarpinski/Downloads/mastar-goodspec-v2_4_3-v1_0_2.fits")[1].data
mastarall = fits.open('/Users/josephkarpinski/Downloads/mastarall-v2_4_3-v1_0_2.fits')

ksel = np.logical_and(mastarall[1].data["INPUT_TEFF"] >= -9999 , mastarall[1].data["INPUT_TEFF"] < 52000)
kstars = mastarall[1].data[ksel]

for x in range(0, 3326):
#for x in range(0, 25):
    visits = goodspec["MANGAID"] == kstars[x]["MANGAID"]
    wl = goodspec[visits]["WAVE"][0]
    flux = goodspec[visits]["FLUX"][0]
    
    mangaid = kstars[x]["MANGAID"]
    logG = kstars[x]["INPUT_LOGG"]
    teff = kstars[x]["INPUT_TEFF"]
    feh = kstars[x]["INPUT_FE_H"]
    
    tbl =np.column_stack((wl, flux))
    df = pd.DataFrame(tbl, columns=['wl', 'flux'])
    
    peak = 0
    
    if teff > 0:
        peak = 2.898 / teff * 10000000
        teffCalc = 2.898 / peak * 10000000
    else:
        peak = wl[flux.argmax()]
        teffCalc = 2.898 / peak * 10000000
 
    plt.figure(facecolor='white')

    plt.plot(wl,flux, color='r', marker='o')
    #plt.plot(wl,flux, color='r')
    plt.ylabel("flux (1e-17 erg/s/cm$^2$/Angstrom")
    plt.xlabel("wavelength (Angstroms)")

    
    
    plt.title("MaNGAID[" + str(mangaid) + "] File_" +  str(x) + "\n" + 
             " Teff: " + str(teff) + 
             " LogG: " + str(logG) + 
             " FeH: " + str(feh) + "\n" +
             " Teff Calc: " + str(int(teffCalc)) +
             " Peak Wavelength: " + str(int(peak))
             )

    plt.grid()
    
    df1 = df[df['wl'].between(6538, 6555)]
    df1Max = float(df1['flux'].max())
    
    df2 = df[df['wl'].between(6555, 6572)]
    yRange = float(df2['flux'].max())
    yRangeMin = float(df2['flux'].min())
    
    numSum = float(df2['flux'].sum())
    lenBand = df2.count()
    denSum = float(lenBand['flux'] * df1Max)
    
    if denSum != 0:
        idx = round(numSum / denSum, 2)
    else:
        idx = -9.999
        
    df1 = df[df['wl'].between(3900, 9000)]
    
    slope, intercept = np.polyfit(np.log(df1.wl), np.log(df1.flux), 1)
    #slope, intercept = np.polyfit(df1.wl, df1.flux, 1)
                   
    plt.annotate(xy=[6566, yRangeMin + (yRangeMin * .05)], s='H-alpha 6563')
    
    #ax = plt.axes()
    plt.xlim(xmin=6555, xmax=6572)
    plt.ylim(ymin=yRangeMin, ymax=yRange)
    
    plt.xticks([6555, 6555.5, 6556, 6556.5, 6557, 6557.5, 6558, 6558.5, 6559, 6559.5, 6560, 6560.5,
    6561, 6561.5, 6562,  6562.5, 6563, 6563.5, 6564, 6564.5, 6565, 6565.5, 6566, 6566.5, 6567, 6567.5,
    6568, 6568.5, 6569, 6569.5, 6570, 6570.5, 6571, 6571.5, 6572])
    
    xLabels = (6555, 6555.5, 6556, 6556.5, 6557, 6557.5, 6558, 6558.5, 6559, 6559.5, 6560, 6560.5,
    6561, 6561.5, 6562,  6562.5, 6563, 6563.5, 6564, 6564.5, 6565, 6565.5, 6566, 6566.5, 6567, 6567.5,
    6568, 6568.5, 6569, 6569.5, 6570, 6570.5, 6571, 6571.5, 6572)

    plt.xticks(xLabels, rotation=90)
    
    plt.savefig('/Users/josephkarpinski/Downloads/MaStar 6563A Halpha Calc/' + 'Slope[' + str(round(slope, 2)) + ']_'  + 
                'Idx[' + str(idx) + ']_' +
                'MaNGAID[' + str(mangaid) + ']' +
                 '_' + str(x) +  '.png')

    plt.gcf().clear()
    plt.close('all')
    
    
    
    
      # create 2nd line plot
    plt.figure(facecolor='white')
    
    plt.plot(wl,flux, color='r')
    plt.ylabel("flux (1e-17 erg/s/cm$^2$/Angstrom")
    plt.xlabel("wavelength (Angstroms)")
    

    plt.title("MaNGAID[" + str(mangaid) + "] File_" +  str(x) + "\n" + 
             " Teff: " + str(teff) + 
             " LogG: " + str(logG) + 
             " FeH: " + str(feh) + "\n" +
             " Teff Calc: " + str(int(teffCalc)) +
             " Peak Wavelength: " + str(int(peak))
             )

    plt.grid()
    
    df2 = df[df['wl'].between(3622, 10354)]
    yRange = float(df2['flux'].max())
    
    plt.annotate(xy=[3839, yRange * .95], s='CN 3839', rotation=-90)
    plt.annotate(xy=[3889, yRange * .20], s='H-zeta 3889', rotation=-90)
    plt.annotate(xy=[3970, yRange * .20], s='H-epsilon 3970', rotation=-90)
    plt.annotate(xy=[4101, yRange * .20], s='H-delta 4101', rotation=-90)
    plt.annotate(xy=[4340, yRange * .20], s='H-gamma 4340', rotation=-90)
    plt.annotate(xy=[4472, yRange * .20], s='He I 4471.7', rotation=-90)
    plt.annotate(xy=[4861, yRange * .20], s='H-beta 4861', rotation=-90)
    plt.annotate(xy=[5175, yRange * .20], s='Mg I 5175', rotation=-90)
    plt.annotate(xy=[5270, yRange * .20], s='Fe I 5270', rotation=-90)
    plt.annotate(xy=[5411, yRange * .20], s='He II 5411', rotation=-90)
    plt.annotate(xy=[5755, yRange * .20], s='N II 5755', rotation=-90)
    plt.annotate(xy=[5895, yRange * .20], s='D1 Na I 5895', rotation=-90)
    plt.annotate(xy=[6302, yRange * .20], s='OI 6302', rotation=-90)
    plt.annotate(xy=[6365, yRange * .20], s='OI 6365.5', rotation=-90)
    plt.annotate(xy=[6562, yRange * .20], s='H-alpha 6562', rotation=-90)
    plt.annotate(xy=[6890, yRange * .20], s='CN 6890', rotation=-90)
    plt.annotate(xy=[7135, yRange * .20], s='Ar III 7135', rotation=-90)
    plt.annotate(xy=[7775, yRange * .20], s='O I 7775', rotation=-90)
    plt.annotate(xy=[8498, yRange * .95], s='Ca II 8498', rotation=-90)
    plt.annotate(xy=[8542, yRange * .20], s='Ca II 8542', rotation=-90)
    plt.annotate(xy=[8662, yRange * .20], s='Ca II 8662.1', rotation=-90)
    
    #ax = plt.axes()
    plt.xlim(xmin=3622, xmax=10354)
    plt.ylim(ymin=0, ymax=yRange)
    
    plt.xticks([3839, 3889, 3970, 4101, 4340, 4472, 4861, 5175, 5270, 5411, 5755, 5895, 6302,
               6365, 6562, 6562, 6890, 7135, 7500, 7775, 8000, 8498, 8542, 8662, 9000, 9250,
               9500, 9750, 10000, 10250])
    
    xLabels = (3839, 3889, 3970, 4101, 4340, 4472, 4861, 5175, 5270, 5411, 5755, 5895, 6302,
               6365, 6562, 6562, 6890, 7135, 7500, 7775, 8000, 8498, 8542, 8662, 9000, 9250,
               9500, 9750, 10000, 10250)
    
    plt.xticks(xLabels, rotation=90)
    
    plt.savefig('/Users/josephkarpinski/Downloads/MaStar 6563A Halpha Calc/' + 'Slope[' + str(round(slope, 2)) + ']_'  + 
                'Idx[' + str(idx) + ']_' +
                'MaNGAID[' + str(mangaid) + ']' +
                 '_' + str(x) +  '_Full.png')
    
    plt.gcf().clear()
    plt.close('all')
    
    
    #####################################################################################################
    
    
 
    
    
print("Done")
    

KeyboardInterrupt: 